In [1]:
!pip install langchain langgraph langsmith langchain-groq langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is th

In [2]:
groq_api_key = "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"

In [3]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq

@tool
def multiply(a: int, b: int) -> int:
    """This function is responsible for multiplication"""
    return a * b

@tool
def sumX(a: int, b: int) -> int:
    """This function is responsible for addition"""
    return a + b

tools = [multiply, sumX]

llm = ChatGroq(model="openai/gpt-oss-20b", groq_api_key=groq_api_key)
llm_with_tool = llm.bind_tools(tools)

class State(TypedDict):
    messages: Annotated[list, add_messages]

def tool_calling_llm(state: State):
    return {
        "messages": [llm_with_tool.invoke(state["messages"])]
    }

memory = MemorySaver()  # Stores conversation history in RAM

graph_builder = StateGraph(State)
graph_builder.add_node("tool_calling_llm", tool_calling_llm)
graph_builder.add_node("tools", ToolNode(tools))

graph_builder.add_edge(START, "tool_calling_llm")
graph_builder.add_conditional_edges("tool_calling_llm", tools_condition)
graph_builder.add_edge("tools", END)

graph = graph_builder.compile(checkpointer=memory)



In [18]:
# Start a session
config = {"configurable": {"thread_id": "session_4"}}  # thread_id keeps memory per user/session


In [19]:

result1 = graph.invoke({"messages": [("user", "Multiply 3 and 5")]} , config=config)
print(result1["messages"][-1].content)


15


In [20]:
result2 = graph.invoke({"messages": [("user", "Now add 10 to that result")]} , config=config)
print(result2["messages"][-1].content)

25


In [21]:
result2 = graph.invoke({"messages": [("user", "Now add 5 to that result")]} , config=config)
print(result2["messages"][-1].content)

30
